In [1]:
from sklearn.metrics import mean_absolute_percentage_error as mape
from all_functions import *
from darts import TimeSeries
import os
from darts.models import StatsForecastAutoETS
from darts.models import StatsForecastAutoARIMA
import statsmodels.api as sm
from statsmodels.tsa.exponential_smoothing.ets import ETSModel
def percorrer_csv(diretorios, save_dir='./', horizon=5):
    cont = 0
    
    resultado_file = save_dir+'/result_ets.csv'
    train = pd.Series()
    for diretorio in diretorios.split():
        if os.path.isdir(diretorio):
            for root, _, arquivos in os.walk(diretorio):
                for arquivo in arquivos:
                    if arquivo.endswith('.csv'):
                        caminho_arquivo = os.path.join(root, arquivo)
                        try:
                            paths = caminho_arquivo.split('/')
                            file_csv = paths[-1]
                            estado = file_csv.split('_')[2]
                            derivado = file_csv.split('_')[3].replace('.csv', '')
                            cidade = file_csv.split('_')[1]
                            df = pd.read_csv(caminho_arquivo, sep=";")
                            df['timestamp'] = pd.to_datetime(df['timestamp'], format='%Y')

                            df.set_index('timestamp', inplace=True)
                            start_timestamp = '1998-01-01'  # ou qualquer data inicial que você queira
                            index_series = pd.date_range(start=start_timestamp, periods=len(df), freq='Y')
                            # series = pd.Series(series_value, index=index_series)
                            serie_m3 = pd.Series(df['m3'].values, index=index_series)
                            train, test = train_test_stats(serie_m3, horizon)
                            w = len(train)
                            # train_tf, _, _ = rolling_window_series(train, horizon)
                            train_tf = znorm(train)
                            train_darts = TimeSeries.from_series(train_tf)
                            
                            if len(serie_m3) > 12:
                                test_index = test
                                #roda 5 vezes para 5 anos
                                preds_final = []
                                preds_series = pd.Series()
                                train_concat = train
                                # model = StatsForecastAutoETS(model='AAN')
                                # model = StatsForecastAutoARIMA(season_length=1)
                                model = ETSModel(train_tf, trend='add', damped_trend=True, seasonal_periods=1, freq='A').fit(full_output=False, disp=False)


                                predictions = model.forecast(steps=horizon)
                                preds_norm = pd.Series(predictions, index=test.index)
                                # model.fit(train_darts)
                                # result = model.predict(n=horizon)
                                # preds_norm = pd.Series(result.values().flatten().tolist(), index=test.index)
                                preds_real = reverse_transform_norm_preds(preds_norm, train_concat, "normal", w=horizon)
                                preds_final = preds_real.values
                                # for i in range(5):
                                #     model = StatsForecastAutoETS()
                                #     model.fit(train_darts)
                                #     result = model.predict(n=1)
                                #     new_index = test_index.index[0] + pd.DateOffset(years=i)
                                #     new_index = pd.DatetimeIndex([new_index])
                                #     preds_norm = pd.Series(result.values().flatten().tolist(), index=new_index)
                                #     w = len(train_concat)
                                #     preds_real = reverse_transform_norm_preds(preds_norm, train_concat, "normal", w=w)
                                #     train_concat = pd.concat([train_concat, preds_real])
                                #     preds_final.extend(preds_real.values)
                                #     if preds_series.empty:
                                #         preds_series = preds_real
                                #     else:
                                #         preds_series = pd.concat([preds_series, preds_real])
                                    
                                #     #concat train com preds_real
                                #     # train_concat_tf, _, _ = rolling_window_series(train_concat, w)
                                #     train_concat_tf = znorm(train_concat)
                                #     train_darts = TimeSeries.from_series(train_concat_tf)
                                    
                                cont+=1
                                mape_result = mape(test.values, preds_final)
                                pocid_result = pocid(test.values, preds_final)
                                dados = {
                                    'PRODUCT': derivado,
                                    'UF': estado.upper(),
                                    'MODEL': 'ETS',
                                    'PARAMETERS': str({'trend':'add', 'damped_trend':True, 'seasonal_periods':1, 'freq':'A'}),
                                    # 'mape': mape_result,
                                    # 'pocid': pocid_result,
                                    'city': cidade,
                                    'test': [list(test.values)],
                                    'preds': [list(preds_final)],
                                    # 'index_test': [test.index],
                                    # 'index_preds': [preds_series.index]
                                    
                                }
                                df_result = pd.DataFrame(dados)
                                df_result = df_result.reset_index(drop=True)
                                print(cont)

                                if not os.path.exists(resultado_file):
                                    df_result.to_csv(resultado_file, sep=';', index=False, mode='w', header=True)
                                else:
                                    df_result.to_csv(resultado_file, sep=';', index=False, mode='a', header=False)
                        except Exception as e:
                            print(f"Erro ao ler {arquivo}: {e}")
        else:
            print(train)
            print(f"O diretório {diretorio} não existe ou não é uma pasta válida.")




/home/user/miniconda3/envs/lucas/lib/python3.11/site-packages/optuna/study/_optimize.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from optuna import progress_bar as pbar_module


In [4]:
from streamfuels.datasets import DatasetLoader

loader = DatasetLoader()
# result, flag = loader.monthly_sales_state()

df, metadata = loader.read_tsf(path_tsf='./monthly_fuel_sales_by_state.tsf')
df

,series_name,start_timestamp,end_timestamp,state_code,product,series_value
0,T1,1990-01-01,2025-06-01,MA,ethanol,"[7578.939, 11475.757, 6385.067, 10140.217, 594..."
1,T2,1990-01-01,2025-06-01,PI,fuel oil,"[332.7479592, 168.6918367, 109.4561224, 227.63..."
2,T3,1990-01-01,2025-06-01,PE,gasoline-a,"[174.962, 174.962, 151.817, 130.48, 146.045, 1..."
3,T4,1990-01-01,2025-06-01,ES,gasoline-a,"[72.757, 22.008, 28.185, 8.538, 46.206, 19.085..."
4,T5,1990-01-01,2025-06-01,SC,ethanol,"[40668.734, 32451.235, 30350.948, 36121.386, 2..."
...,...,...,...,...,...,...
211,T212,1990-01-01,2025-06-01,PR,diesel,"[153548.404, 144427.716, 171067.003, 234172.8,..."
212,T213,1990-01-01,2025-06-01,AC,LPG,"[1310.456522, 1212.697464, 1268.0, 1557.637681..."
213,T214,1990-01-01,2025-06-01,MG,kerosene-a,"[8564.399, 6235.286, 4334.646, 7481.727, 6199...."
214,T215,1990-01-01,2025-06-01,GO,kerosene-i,"[368.413, 415.717, 420.841, 731.662, 455.111, ..."
